In [35]:
library(dplyr)
my_db <- src_sqlite("database.sqlite", create = FALSE)

In [37]:
my_db

src:  sqlite 3.11.1 [/Users/arunchandrapendyala/Desktop/DIC/lab2/database.sqlite]
tbls: Country, League, Match, Player, Player_Attributes, sqlite_sequence, Team,
  Team_Attributes

In [38]:
player_sqlite <- tbl(my_db,"player")
playattr_sqlite <- tbl(my_db , "Player_Attributes")
league_sqlite <- tbl(my_db, "League")
teamattr_sqlite <- tbl(my_db,"Team_Attributes")
match_sqlite <- tbl(my_db,"Match")
team_sqlite <- tbl(my_db,"Team")


In [39]:
head(player_sqlite)

Source:   query [?? x 7]
Database: sqlite 3.11.1 [/Users/arunchandrapendyala/Desktop/DIC/lab2/database.sqlite]

     id player_api_id        player_name player_fifa_api_id            birthday
  <int>         <int>              <chr>              <int>               <chr>
1     1        505942 Aaron Appindangoye             218353 1992-02-29 00:00:00
2     2        155782    Aaron Cresswell             189615 1989-12-15 00:00:00
3     3        162549        Aaron Doran             186170 1991-05-13 00:00:00
4     4         30572      Aaron Galindo             140161 1982-05-08 00:00:00
5     5         23780       Aaron Hughes              17725 1979-11-08 00:00:00
6     6         27316         Aaron Hunt             158138 1986-09-04 00:00:00
# ... with 2 more variables: height <dbl>, weight <int>

# Creating Player.csv

In [40]:
#obtaining the gk rating as sum of 5 gk attributes
#by_playattr <- group_by(playattr_sqlite,player_id)
new_playattr <- mutate(playattr_sqlite, gk_rate = gk_diving + gk_handling + gk_kicking + gk_positioning +gk_reflexes )
by_playid <- select(new_playattr,player_id = player_api_id,overall_rating,gk_rate) %>% group_by(player_id) 

In [41]:
play_gk_ovr <- summarise(by_playid, overall_rate = max(overall_rating),gk_rate)


head(as.data.frame(play_gk_ovr),n=10) #gk_rating and overall_rating ----> (1)

player_id,overall_rate,gk_rate
2625,63,153
2752,72,53
2768,75,64
2770,74,60
2790,73,144
2796,69,340
2802,82,54
2805,76,56
2857,72,143
2862,76,44


In [42]:
#select the player appearances in matches as home and away players
players <- select(match_sqlite,home_player_1:away_player_11,league_id)%>% as.data.frame()


In [43]:
head(players)


home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,⋯,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,league_id
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,1
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,1
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,1
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,1
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,1
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,1


In [44]:
#convert all columns into a single column to get the number of appearances
playlist <- cbind(players[23],stack(players[1:22])) #source:https://stat.ethz.ch/R-manual/R-devel/library/utils/html/stack.html
#source: http://stackoverflow.com/questions/13597091/stacking-columns-in-data-frame-into-one-column-in-r

In [45]:
head(playlist)

league_id,values,ind
1,NA,home_player_1
1,NA,home_player_1
1,NA,home_player_1
1,NA,home_player_1
1,NA,home_player_1
1,NA,home_player_1


In [46]:
by_values <- group_by(playlist,player_id = values)
play_num <- summarise(by_values, num_appear = n()) %>% na.omit()
#num of appearances ---> (2)

In [47]:
head(play_num,n=10)

player_id,num_appear
2625,90
2752,81
2768,71
2770,16
2790,50
2796,1
2802,144
2805,131
2857,3
2862,17


In [48]:
by_league <- group_by(playlist , player_id = values)
leag_num <- summarise(by_league, leag_num = n_distinct(league_id))
head(arrange(leag_num,desc(leag_num)))
#number of leagues for each player ----> (3)

player_id,leag_num
NA,11
42119,6
116750,5
178538,5
22543,4
24123,4


In [49]:
#joining player-id, number of appearances and number of leagues
appear_leag <- play_num %>% left_join(leag_num, by = "player_id") 
appear_leag  #-----> (4) obtained by merging (2) & (3)

player_id,num_appear,leag_num
2625,90,1
2752,81,1
2768,71,1
2770,16,2
2790,50,1
2796,1,1
2802,144,2
2805,131,1
2857,3,1
2862,17,1


In [50]:
play_info <- select(player_sqlite, player_id = player_api_id, player_name)
head(as.data.frame(play_info))  #player id and player name ----> (5)

player_id,player_name
505942,Aaron Appindangoye
155782,Aaron Cresswell
162549,Aaron Doran
30572,Aaron Galindo
23780,Aaron Hughes
27316,Aaron Hunt


In [51]:
#merging (1) , (4), (5)

part_play_info <- play_info %>% left_join(play_gk_ovr, by = "player_id") 
full_player_info <- part_play_info%>% left_join(appear_leag,by="player_id",copy = TRUE)


In [52]:
head(as.data.frame(full_player_info))

player_id,player_name,overall_rate,gk_rate,num_appear,leag_num
505942,Aaron Appindangoye,67,43,8,1
155782,Aaron Cresswell,74,51,75,1
162549,Aaron Doran,71,56,104,1
30572,Aaron Galindo,75,111,10,1
23780,Aaron Hughes,78,129,162,1
27316,Aaron Hunt,79,60,158,1


In [53]:
full_play_df <- as.data.frame(full_player_info)
write.csv(na.omit(full_play_df), file = "player.csv",row.names=FALSE, na="")

# Creating team.csv

In [54]:
#obtaining the buildupplay , chance creation and defence as sum of its attributes
#by_teamid <- group_by(teamattr_sqlite,team_id)
sums_teamattr <- mutate(teamattr_sqlite, buildup_sum = buildUpPlaySpeed + buildUpPlayDribbling + buildUpPlayPassing, chance_sum = chanceCreationPassing + chanceCreationCrossing + chanceCreationShooting, defence_sum = defencePressure + defenceAggression + defenceTeamWidth )
by_teamid <- select(sums_teamattr,team_id = team_api_id,buildup_sum,chance_sum,defence_sum) %>% group_by(team_id)  


In [55]:
#use max() function to get highest attribute value to resolve ties
new_teamattr <- summarise(by_teamid ,buildup = max(buildup_sum),chance= max(chance_sum), defence = max(defence_sum))
new_teamattr <- na.omit(new_teamattr)

In [56]:
head(as.data.frame(new_teamattr),n=10)

team_id,buildup,chance,defence
1601,139,186,175
1773,153,157,137
1957,158,163,210
2033,147,165,155
2182,176,196,160
2183,NA,180,175
2186,125,168,205
4087,156,180,165
4170,NA,165,200
6269,179,172,151


In [57]:
home_goal <- select(match_sqlite,team_id = home_team_api_id, goals = home_team_goal) %>% as.data.frame()
away_goal <- select(match_sqlite,team_id = away_team_api_id, goals = away_team_goal) %>% as.data.frame()

In [58]:
goals_df <- rbind(home_goal,away_goal)
head(goals_df) #goals obtained by considering home_team_goals and away_team_goals


team_id,goals
9987,1
10000,0
9984,0
9991,5
7947,1
8203,1


In [59]:
goals_sum <- group_by(goals_df,team_id) %>% summarise(goals_total = sum(goals)) 
head(goals_sum)

team_id,goals_total
1601,280
1773,126
1957,287
2033,138
2182,372
2183,184


In [60]:
teamattr_df <- select(new_teamattr,team_id,buildup,chance,defence) %>% as.data.frame()
team_df <- select(team_sqlite,team_id = team_api_id,team_name = team_long_name) %>% as.data.frame()


In [61]:
full_team_df <- team_df %>% left_join(teamattr_df, by = "team_id") %>% left_join(goals_sum, by= "team_id")

In [63]:
write.csv(full_team_df, file = "team.csv",row.names=FALSE, na="")